In [ ]:
import os

from dotenv import load_dotenv

load_dotenv()
import psycopg
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import mlflow

In [21]:
import os

import pandas as pd
import mlflow
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (
    OneHotEncoder, 
    SplineTransformer, 
    QuantileTransformer, 
    RobustScaler,
    PolynomialFeatures,
    KBinsDiscretizer,
)

TABLE_NAME = "users_churn" # таблица с данными

TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5001

EXPERIMENT_NAME = "churn_prediction_experiment_budikdb" # напишите название вашего эксперимента
RUN_NAME = "preprocessing" 
REGISTRY_MODEL_NAME ="churn_model_budikdb_b2c" # название зарегистрированной модели

In [22]:
connection = {
    "sslmode": "require",
    "target_session_attrs": "read-write",
    "connect_timeout": 10
}
postgres_credentials = {
    "host": os.getenv("DB_DESTINATION_HOST"),
    "port": os.getenv("DB_DESTINATION_PORT"),
    "dbname": os.getenv("DB_DESTINATION_NAME"),
    "user": os.getenv("DB_DESTINATION_USER"),
    "password": os.getenv("DB_DESTINATION_PASSWORD"),
}

connection.update(postgres_credentials)

with psycopg.connect(**connection) as conn:

    with conn.cursor() as cur:
        cur.execute(f"SELECT * FROM {TABLE_NAME}")
        data = cur.fetchall()
        columns = [col[0] for col in cur.description]

df = pd.DataFrame(data, columns=columns)

df.head(2)

,id,customer_id,begin_date,end_date,type,paperless_billing,payment_method,monthly_charges,total_charges,internet_service,...,device_protection,tech_support,streaming_tv,streaming_movies,gender,senior_citizen,partner,dependents,multiple_lines,target
0,12,7469-LKBCI,2018-10-01,NaT,Two year,No,Credit card (automatic),18.95,326.8,None,...,None,None,None,None,Male,0,No,No,No,0
1,13,8091-TTVAX,2015-04-01,NaT,One year,No,Credit card (automatic),100.35,5681.1,Fiber optic,...,Yes,No,Yes,Yes,Male,0,Yes,No,Yes,0


In [23]:
obj_df = df.select_dtypes(include="object")
# определение категориальных колонок, которые будут преобразованы
cat_columns = ["type", "payment_method", "internet_service", "gender"]

# создание объекта OneHotEncoder для преобразования категориальных переменных
encoder_oh = OneHotEncoder(categories='auto', handle_unknown='ignore', max_categories=10, sparse_output=False, drop='first')

# применение OneHotEncoder к данным. Преобразование категориальных данных в массив
encoded_features = encoder_oh.fit_transform(df[cat_columns].to_numpy())

# преобразование полученных признаков в DataFrame и установка названий колонок
encoded_df = pd.DataFrame(
    encoded_features,
    columns=encoder_oh.get_feature_names_out(cat_columns)
)

# конкатенация исходного DataFrame с новым DataFrame, содержащим закодированные категориальные признаки
obj_df = pd.concat([obj_df, encoded_df], axis=1)

obj_df.head(2)

,customer_id,type,paperless_billing,payment_method,internet_service,online_security,online_backup,device_protection,tech_support,streaming_tv,...,dependents,multiple_lines,type_One year,type_Two year,payment_method_Credit card (automatic),payment_method_Electronic check,payment_method_Mailed check,internet_service_Fiber optic,internet_service_None,gender_Male
0,7469-LKBCI,Two year,No,Credit card (automatic),None,None,None,None,None,None,...,No,No,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0
1,8091-TTVAX,One year,No,Credit card (automatic),Fiber optic,No,No,Yes,No,Yes,...,No,Yes,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0


In [24]:
num_columns = ["monthly_charges", "total_charges"]

n_knots = 3
degree_spline = 4
n_quantiles=100
degree = 3
n_bins = 5
encode = 'ordinal'
strategy = 'uniform'
subsample = None

df[num_columns] = df[num_columns].fillna(df[num_columns].mean())
# SplineTransformer
encoder_spl = SplineTransformer(n_knots=n_knots, degree=degree_spline)
encoded_features = encoder_spl.fit_transform(df[num_columns].to_numpy())

encoded_df = pd.DataFrame(
    encoded_features, 
    columns=encoder_spl.get_feature_names_out(num_columns)
)
num_df = pd.concat([df[num_columns], encoded_df], axis=1)


# QuantileTransformer
encoder_q = QuantileTransformer(n_quantiles=n_quantiles)
encoded_features = encoder_q.fit_transform(df[num_columns].to_numpy())

encoded_df = pd.DataFrame(encoded_features, columns=encoder_q.get_feature_names_out(num_columns))
encoded_df.columns = [col + f"_q_{n_quantiles}" for col in num_columns]
num_df = pd.concat([df[num_columns], encoded_df], axis=1)


# RobustScaler
encoder_rb = RobustScaler()
encoded_features = encoder_rb.fit_transform(df[num_columns].to_numpy())

encoded_df = pd.DataFrame(encoded_features, columns=encoder_rb.get_feature_names_out(num_columns))
encoded_df.columns = [col + f"_robust" for col in num_columns]
num_df = pd.concat([df[num_columns], encoded_df], axis=1)


# PolynomialFeatures
encoder_pol = PolynomialFeatures(degree=degree)
encoded_features = encoder_pol.fit_transform(df[num_columns].to_numpy())

poly_names = encoder_pol.get_feature_names_out(num_columns)
encoded_df = pd.DataFrame(encoded_features, columns=poly_names)

# Пропускаем 1 (intercept) + 2 (исходных признака)
encoded_df = encoded_df.iloc[:, 1 + len(num_columns):]

# Генерируем корректные имена
new_names = poly_names[1 + len(num_columns):]
encoded_df.columns = [name + "_poly" for name in new_names]
num_df = pd.concat([df[num_columns], encoded_df], axis=1)

# KBinsDiscretizer
encoder_kbd = KBinsDiscretizer(n_bins=n_bins, encode=encode, strategy=strategy, subsample=subsample)
encoded_features = encoder_kbd.fit_transform(df[num_columns].to_numpy())

encoded_df = pd.DataFrame(encoded_features, columns=encoder_kbd.get_feature_names_out(num_columns))
encoded_df.columns = [col + f"_bin" for col in num_columns]
num_df = pd.concat([df[num_columns], encoded_df], axis=1)


num_df.head(2)

,monthly_charges,total_charges,monthly_charges_bin,total_charges_bin
0,18.95,326.8,0.0,0.0
1,100.35,5681.1,4.0,3.0


In [25]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import SplineTransformer, QuantileTransformer, RobustScaler, PolynomialFeatures, KBinsDiscretizer, OneHotEncoder
from sklearn.impute import SimpleImputer

# Предполагаем, что у нас есть следующие переменные, определенные ранее:
# num_columns = ["monthly_charges", "total_charges"]
# cat_columns = ["type", "payment_method", "internet_service", "gender"]
# df - исходный DataFrame

numeric_transformer = ColumnTransformer(transformers=[('spl', encoder_spl, num_columns), ('q', encoder_q, num_columns), ('rb', encoder_rb, num_columns), ('pol', encoder_pol, num_columns), ('kbd', encoder_kbd, num_columns)])

categorical_transformer = Pipeline(steps=[('encoder', encoder_oh)])

preprocessor = ColumnTransformer(transformers=[('num', numeric_transformer, num_columns), ('cat', categorical_transformer, cat_columns)], n_jobs=-1)
encoded_features = preprocessor.fit_transform(df)

# Получаем имена столбцов для преобразованных данных
feature_names = preprocessor.get_feature_names_out()

transformed_df = pd.DataFrame(encoded_features, columns=preprocessor.get_feature_names_out())

# Объединяем с исходным датасетом
df = pd.concat([df, transformed_df], axis=1)

df.head(2)

,id,customer_id,begin_date,end_date,type,paperless_billing,payment_method,monthly_charges,total_charges,internet_service,...,num__kbd__monthly_charges,num__kbd__total_charges,cat__type_One year,cat__type_Two year,cat__payment_method_Credit card (automatic),cat__payment_method_Electronic check,cat__payment_method_Mailed check,cat__internet_service_Fiber optic,cat__internet_service_None,cat__gender_Male
0,12,7469-LKBCI,2018-10-01,NaT,Two year,No,Credit card (automatic),18.95,326.8,None,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0
1,13,8091-TTVAX,2015-04-01,NaT,One year,No,Credit card (automatic),100.35,5681.1,Fiber optic,...,4.0,3.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0


In [26]:
preprocessor

ColumnTransformer(n_jobs=-1,
                  transformers=[('num',
                                 ColumnTransformer(transformers=[('spl',
                                                                  SplineTransformer(degree=4,
                                                                                    n_knots=3),
                                                                  ['monthly_charges',
                                                                   'total_charges']),
                                                                 ('q',
                                                                  QuantileTransformer(n_quantiles=100),
                                                                  ['monthly_charges',
                                                                   'total_charges']),
                                                                 ('rb',
                                                                  RobustScaler(),
                                                                  ['monthly_charges',
                                                                   'total_charges']),
                                                                 ('pol',
                                                                  PolynomialFeatures(degree=3),
                                                                  ['monthly_char...
                                                                   'total_charges']),
                                                                 ('kbd',
                                                                  KBinsDiscretizer(encode='ordinal',
                                                                                   strategy='uniform',
                                                                                   subsample=None),
                                                                  ['monthly_charges',
                                                                   'total_charges'])]),
                                 ['monthly_charges', 'total_charges']),
                                ('cat',
                                 Pipeline(steps=[('encoder',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                max_categories=10,
                                                                sparse_output=False))]),
                                 ['type', 'payment_method', 'internet_service',
                                  'gender'])])

In [ ]:
os.environ["MLFLOW_S3_ENDPOINT_URL"] =  "***"
os.environ["AWS_ACCESS_KEY_ID"] = "***"
os.environ["AWS_SECRET_ACCESS_KEY"] = "***"

mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")

experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id

    mlflow.sklearn.log_model(preprocessor, artifact_path="column_transformer")

2025/09/16 20:48:42 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/16 20:48:43 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2025/09/16 20:48:46 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run preprocessing at: http://127.0.0.1:5001/#/experiments/5/runs/4454bd8c41844597ba8972059aa2d15c
🧪 View experiment at: http://127.0.0.1:5001/#/experiments/5


S3UploadFailedError: Failed to upload /tmp/tmp13j6a8r1/model/python_env.yaml to s3-student-mle-20250130-d1608e0ec6/5/models/m-ceb582744767447582890258fc2f5097/artifacts/python_env.yaml: An error occurred (SignatureDoesNotMatch) when calling the PutObject operation: The request signature we calculated does not match the signature you provided. Check your key and signing method.

In [ ]:
mlflow.set_tracking_uri("http://127.0.0.1:5001")
mlflow.set_experiment("churn_prediction_experiment_budikdb")

# Путь к артефактам можно получить внутри run
with mlflow.start_run(run_name="preprocessing") as run:
    artifact_uri = mlflow.get_artifact_uri()
    print("Artifacts saved to:", artifact_uri)

Artifacts saved to: s3://s3-student-mle-20250130-d1608e0ec6/5/3abfc5e1fc5444cb920e14d24b180672/artifacts
🏃 View run preprocessing at: http://127.0.0.1:5001/#/experiments/5/runs/3abfc5e1fc5444cb920e14d24b180672
🧪 View experiment at: http://127.0.0.1:5001/#/experiments/5
